# TDVP of Bose-Hubbard model 

The Hamiltonian is $H=-J\sum_{j=1}^{N-1}(a_j^{\dagger}a_{j+1}+a_{j+1}^{\dagger}a_{j})+\frac{U}{2}\sum_{j=1}^Nn_j(n_j-1)$

In [ ]:
using ITensors
using Plots
using LinearAlgebra
using ITensorMPS
using LaTeXStrings
plot_font = "Computer Modern";

Setup parameters

In [ ]:
N = 4   # Number of sites  
nmax = 3   # Maximum number of bosons per site

J = 1.0   # Hopping
U = 2.0   # Interaction

T = 2.0  # Final time
δt = 0.01   # Time step
time = 0.0:δt:T   # Time vector
tbigstep = 10   # Calculate expectation values each tbigstep times
num_expvals = Int(((length(time)-1)/tbigstep)) + 1; # Number of times expectation values will be calculated. The ±1 is to account correctly for t = 0  
cutoff = 1E-8;   # Truncation allowed per step
χ = 100; # Maximum bond dimension

Initialize expectation values, namely boson population and coherences $\langle a_i^{\dagger}a_j\rangle$ 

In [ ]:
Norm = zeros(num_expvals,1); # Norm of evolved state
Popul = zeros(num_expvals,N); # Number of bosons per site
Coherences = zeros(num_expvals,N,N)+1im*zeros(num_expvals,N,N); # Coherences between all sites, can be complex
Time_expvals = zeros(num_expvals,1); # Time of expectation values

Define bosonic operators according to nmax

In [ ]:
function ITensors.space(::SiteType"MyBoson";
                        conserve_qns=true)
  if conserve_qns
       
    # Define array of pairs of quantum numbers and the dimension of each one   
    array = [QN("nb",0)=>1];    # Initialize array of pairs
    for k = 1:nmax
        append!(array, [QN("nb",k)=>1])
    end
        
    return array       
  end
    
  return nmax+1 # Only return full dimension if no quantum numbers are used
end

function ITensors.op!(Op::ITensor,
                      ::OpName"Num",
                      ::SiteType"MyBoson",
                      s::Index)    
    # Fill diagonal 
    for k = 1:nmax
        Op[s'=>k+1,s=>k+1] = k
    end
    
end

function ITensors.op!(Op::ITensor,
                      ::OpName"Num2",
                      ::SiteType"MyBoson",
                      s::Index)    
    # Fill diagonal 
    for k = 1:nmax
        Op[s'=>k+1,s=>k+1] = k*k
    end
    
end

function ITensors.op!(Op::ITensor,
                      ::OpName"a",
                      ::SiteType"MyBoson",
                       s::Index)
    # Fill +1 diagonal
    for k = 1:nmax
       Op[s'=>k,s=>k+1] = sqrt(k);
    end
    
end

function ITensors.op!(Op::ITensor,
                      ::OpName"adag",
                      ::SiteType"MyBoson",
                      s::Index)      
    # Fill -1 diagonal    
    for k = 1:nmax
       Op[s'=>k+1,s=>k] = sqrt(k);
    end    
    
end

function ITensors.op!(Op::ITensor,
                      ::OpName"Iden",
                      ::SiteType"MyBoson",
                      s::Index)    
    # Fill diagonal 
    for k = 1:nmax
        Op[s'=>k,s=>k] = 1
    end
    
end

Function to define initial state

In [ ]:
function InitialState(s,N)

     state = [1 for n=1:N] # Initial empty lattice

    for i = 1:N
       if mod(i,2) == 0
           state[i] = 2 # Put boson on even sites
       end
    end

#     for i = Int(0.25*N):Int(0.75*N)
#         state[i] = 2 # Put boson on sites in first half
#     end
    
    ψ0 = MPS(s,state);
    
    return ψ0;
end

Function to define MPO of Hamiltonian

In [ ]:
function Hamiltonian(N,sites,J,U)

    # Initiate construction
    ops = OpSum()
    
    # Hopping terms
    for j=1:N-1
        ops += -J,"a",j,"adag",j+1
        ops += -J,"adag",j,"a",j+1     
    end
    
    # Interaction
    for j=1:N
        ops += 0.5*U,"Num2",j;
        ops += -0.5*U,"Num",j;     
    end

    H = MPO(ops,sites);
    
    return H;
    
end;

Function to calculate expectation values

In [ ]:
function ExpVals!(ψ,N,Popul,Coherences,Norm,timeval) # Here, timeval is the position in the arrays of expectation values where info will be stored 

   # The functions expect and correlation_matrix normalize internally the expectation values
   Popul[timeval,:] = real(expect(ψ, "Num")); # Population of each site
   Coherences[timeval,:,:] = correlation_matrix(ψ,"adag","a"); # Single particle density matrix
   Norm[timeval,1] = real(norm(ψ)); # Norm of the state
    
end;

----------- Main code of time evolution -----------

Define index, and specify use of quantum numbers

In [ ]:
s = siteinds("MyBoson", N, conserve_qns=true); # For all sites

Define initial state

In [ ]:
ψ0 = InitialState(s,N);

Calculate initial expectation values

In [ ]:
ExpVals!(ψ0, N, Popul, Coherences, Norm, 1);

@show Popul[1,:];

Define time evolution MPO

In [ ]:
H = Hamiltonian(N,s,J,U);

Perform time evolution using TDVP

In [ ]:
ψ = ψ0

count_expvals = 1;

for t in 1:length(time)-1
    
    ψ = tdvp(H, -im * δt, ψ; time_step = -im * δt, cutoff = cutoff, maxdim = χ, outputlevel=0, normalize=false);
    
    #normalize!(ψ)
    
    if(mod(t,tbigstep)== 0)
       
        println("Calculating expectation values for $(t) number of steps")
        count_expvals = count_expvals + 1;

        Time_expvals[count_expvals] = t*δt;
            
        # The state is not normalized, just the expectation values in the function
        ExpVals!(ψ, N, Popul, Coherences, Norm, count_expvals);                

    end
    
end;

Plot results

In [ ]:
j_array = 1:N
heatmap(Time_expvals[:,1], j_array, Popul', xlabel = "Time", ylabel = "j", xtickfontsize = 15, ytickfontsize = 15, xguidefontsize = 15, yguidefontsize = 15, colorbar_tickfontsize = 15, c = :plasma, fontfamily=plot_font)

In [ ]:
plot(Time_expvals,Popul[:,Int(0.5*N)], xlabel = "Time", ylabel = "Population at site N/2", xtickfontsize = 15, ytickfontsize = 15, xguidefontsize = 15, yguidefontsize = 15, colorbar_tickfontsize = 15, c = :plasma, fontfamily=plot_font)

In [ ]:
plot(Time_expvals,real(Coherences[:,1,Int(0.5*N)]), xlabel = "Time", ylabel = "Coherence sites 1 and N/2", xtickfontsize = 15, ytickfontsize = 15, xguidefontsize = 15, yguidefontsize = 15, colorbar_tickfontsize = 15, c = :plasma, fontfamily=plot_font)

Simulation finished!!!